## Bracket Challenge Predictor

Reading data

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import re

from functools import reduce
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier

## Data Preprocessing

In [2]:
team_matchups = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/bracketology/data/Tournament Matchups.csv")

away_stats = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/bracketology/data/Barttorvik Away.csv")
home_stats = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/bracketology/data/Barttorvik Home.csv")
neutral_stats = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/bracketology/data/Barttorvik Neutral.csv")

conf_perf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/bracketology/data/Conference Results.csv")

team_res = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/bracketology/data/Team Results.csv")
team_hist = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/bracketology/data/Resumes.csv")
team_v_ranked = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/bracketology/data/TeamRankings.csv")
team_mis = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/bracketology/data/KenPom Barttorvik.csv")

ken_conf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/bracketology/data/DEV _ March Madness.csv")

cur_coaches = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/bracketology/data/REF _ Current NCAAM Coaches.csv")
coach_perf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/bracketology/data/Coach Results.csv")

seed_res = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/bracketology/data/Seed Results.csv")
upset_count = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/bracketology/data/Upset Count.csv")
upset_inf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/bracketology/data/Upset Seed Info.csv")

/var/folders/g4/t1q9yqqx7h36p5_2jzq23jsw0000gn/T/ipykernel_15715/771547425.py:14: DtypeWarning: Columns (144) have mixed types. Specify dtype option on import or set low_memory=False.
  ken_conf = pd.read_csv("/Users/nicholaskim/Documents/Repositories/bracket challenge/bracketology/data/DEV _ March Madness.csv")


since ken_ datasets don't have TEAMNO var will first create seperate lookup table to create a column for its respective id

In [3]:
team_ids = team_matchups[['YEAR','TEAM','TEAM NO']].drop_duplicates()

team_res.rename(columns={'TEAM ID':'TEAM NO'},inplace=True)

In [4]:
temp_away = away_stats.loc[:,['YEAR','TEAM NO','BADJ O','BADJ D','WIN%','EFG%','EFG%D','PPPO','PPPD']]
temp_home = home_stats.loc[:,['YEAR','TEAM NO','BADJ O','BADJ D','WIN%','EFG%','EFG%D','PPPO','PPPD']]
temp_neutral = neutral_stats.loc[:,['YEAR','TEAM NO','BADJ O','BADJ D','WIN%','EFG%','EFG%D','PPPO','PPPD']]

#adds _A to all columns to represent Away
temp_away.columns = list(temp_away.columns[:2]) + [col + "_A" for col in temp_away.columns[2:]]

#_H for Home
temp_home.columns = list(temp_home.columns[:2]) + [col + "_H" for col in temp_home.columns[2:]]

#_N for Neutral
temp_neutral.columns = list(temp_neutral.columns[:2]) + [col + "_N" for col in temp_neutral.columns[2:]]


#combined stats as they are highly correlated away, home, and neutral
#function to create new combined dataframe easily
def var_mean(dfs, vars):
    combined_df = [dfs[0].loc[:,vars],dfs[1].loc[:,vars],dfs[2].loc[:,vars]]
    combined_df = reduce(lambda left, right: pd.merge(left, right, on=['YEAR','TEAM NO']),combined_df)
    for var in vars[2:]:
        combined_df[var] = combined_df[[var+'_x',var+'_y',var]].mean(axis=1)
        combined_df.drop([var+'_x',var+'_y'],axis=1,inplace=True)
    return combined_df

vars = ['YEAR','TEAM NO','BADJ EM','FTR','FTRD',
        'BADJ T','TOV%','TOV%D','OREB%','OP OREB%','RAW T','WAB',
        '2PT%', '2PT%D', '3PT%', '3PT%D', 'BLK%', 'BLKED%',
        'AST%', 'OP AST%', '2PTR', '3PTR', '2PTRD', '3PTRD','ELITE SOS']
dfs = [away_stats,home_stats,neutral_stats]

team_stats = var_mean(dfs,vars)

temp_res = team_res.loc[:,['TEAM','PAKE','PASE','R64','R32','S16','E8','F4',
                           'F2','CHAMP','TOP2']]

temp_his = team_hist.drop(['TEAM','SEED','ROUND','Q1 PLUS Q2 W','B POWER','BID TYPE'],axis=1)

temp_v_ranked = team_v_ranked.loc[:,['YEAR','TEAM NO','TR RATING','V 1-25 WINS','V 1-25 LOSS',
                                     'V 26-50 WINS','V 26-50 LOSS','LUCK RATING','CONSISTENCY TR RATING']]

temp_mis = team_mis.loc[:,['YEAR','CONF','CONF ID','TEAM NO','K OFF','K DEF','AVG HGT','EFF HGT','EXP','TALENT']]


Created a column on who won each matchup for every game in the dataset. Will now add columns of interest to this dataset and will try to do feature engineering as to model who'd win each matchup.

In [5]:
wins = []
for i in range(0,len(team_matchups)-1,2):
    if team_matchups.iloc[i,8] > team_matchups.iloc[i+1,8]:
        wins.append(1)
        wins.append(0)
    else:
        wins.append(0)
        wins.append(1)
    
comp_stats = team_matchups.iloc[:,[0,3,4,5,6,7,8]]
comp_stats['WIN'] = wins
        
comp_stats.head()

/var/folders/g4/t1q9yqqx7h36p5_2jzq23jsw0000gn/T/ipykernel_15715/1728178133.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_stats['WIN'] = wins


,YEAR,TEAM NO,TEAM,SEED,ROUND,CURRENT ROUND,SCORE,WIN
0,2024,1067,Connecticut,1,1,64,91,1
1,2024,1026,Stetson,16,64,64,52,0
2,2024,1060,Florida Atlantic,8,64,64,65,0
3,2024,1036,Northwestern,9,32,64,77,1
4,2024,1029,San Diego St.,5,16,64,69,1


Creating matchup id which will be useful when performing train test split as we need the games split together

In [6]:
matchup_id = []

for i in range(len(comp_stats)//2):
    matchup_id.append(i)
    matchup_id.append(i)

comp_stats['MATCHUP ID'] = matchup_id
comp_stats = comp_stats.iloc[:,[0,1,2,8,3,4,5,6,7]]
comp_stats

,YEAR,TEAM NO,TEAM,MATCHUP ID,SEED,ROUND,CURRENT ROUND,SCORE,WIN
0,2024,1067,Connecticut,0,1,1,64,91,1
1,2024,1026,Stetson,0,16,64,64,52,0
2,2024,1060,Florida Atlantic,1,8,64,64,65,0
3,2024,1036,Northwestern,1,9,32,64,77,1
4,2024,1029,San Diego St.,2,5,16,64,69,1
...,...,...,...,...,...,...,...,...,...
2009,2008,43,Kansas,1004,1,1,4,84,1
2010,2008,37,Memphis,1005,1,2,4,78,1
2011,2008,13,UCLA,1005,1,4,4,63,0
2012,2008,43,Kansas,1006,1,1,2,75,1


In [7]:
comp_team_stats = comp_stats.merge(temp_mis,on=['YEAR','TEAM NO'],how='left')

#rearranging so CONF and CONF ID are near front
comp_team_stats = comp_team_stats.iloc[:,[0,1,2,9,10,3,4,5,6,7,8,11,12,13,14]]

dfs = [comp_team_stats.drop(['SCORE'],axis=1),temp_away,temp_home,temp_neutral,temp_v_ranked,temp_his,team_stats]

comp_team_stats = reduce(lambda left, right: pd.merge(left, right, on=['YEAR','TEAM NO']),dfs)

comp_team_stats = comp_team_stats.merge(temp_res,on='TEAM',how='left')

comp_team_stats.head()

,YEAR,TEAM NO,TEAM,CONF,CONF ID,MATCHUP ID,SEED,ROUND,CURRENT ROUND,WIN,...,PAKE,PASE,R64,R32,S16,E8,F4,F2,CHAMP,TOP2
0,2024,1067,Connecticut,BE,8,0,1,1,64,1,...,10.8,13.3,10.0,6.0,5.0,5.0,5.0,4.0,4.0,2.0
1,2024,1026,Stetson,ASun,5,0,16,64,64,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024,1060,Florida Atlantic,Amer,4,1,8,64,64,0,...,2.8,2.7,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
3,2024,1036,Northwestern,B10,6,1,9,32,64,1,...,0.9,0.8,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024,1029,San Diego St.,MWC,20,2,5,16,64,1,...,0.8,1.1,11.0,6.0,4.0,1.0,1.0,1.0,0.0,1.0


In [10]:
len(comp_team_stats),len(team_matchups)

(2014, 2014)

## Feature engineering

In [8]:
def differenced_matchup(df):
    matchup = []
    for i in range(0,len(df)-1,2):
        team_a = df.iloc[i,:]
        team_b = df.iloc[i+1,:]
    
        if team_a['WIN'] == 1:
            diff = team_a.iloc[10:]-team_b.iloc[10:]
            row = list(team_a.iloc[0:10]) + list(diff)
            matchup.append(row)
            
            row = list(team_b.iloc[0:10]) + list(-diff)
            matchup.append(row)
        else:
            diff = team_b.iloc[10:]-team_a.iloc[10:]
            row = list(team_b.iloc[0:10]) + list(diff)
            matchup.append(row)
            
            row = list(team_a.iloc[0:10]) + list(-diff)
            matchup.append(row)

    columnss = list(df.columns)
    return pd.DataFrame(matchup,columns=columnss) 

diff_team_stats = differenced_matchup(comp_team_stats)
diff_team_stats

,YEAR,TEAM NO,TEAM,CONF,CONF ID,MATCHUP ID,SEED,ROUND,CURRENT ROUND,WIN,...,PAKE,PASE,R64,R32,S16,E8,F4,F2,CHAMP,TOP2
0,2024,1067,Connecticut,BE,8,0,1,1,64,1,...,10.8,13.3,9.0,6.0,5.0,5.0,5.0,4.0,4.0,2.0
1,2024,1026,Stetson,ASun,5,0,16,64,64,0,...,-10.8,-13.3,-9.0,-6.0,-5.0,-5.0,-5.0,-4.0,-4.0,-2.0
2,2024,1036,Northwestern,B10,6,1,9,32,64,1,...,-1.9,-1.9,1.0,2.0,-1.0,-1.0,-1.0,0.0,0.0,0.0
3,2024,1060,Florida Atlantic,Amer,4,1,8,64,64,0,...,1.9,1.9,-1.0,-2.0,1.0,1.0,1.0,-0.0,-0.0,-0.0
4,2024,1029,San Diego St.,MWC,20,2,5,16,64,1,...,0.5,1.3,8.0,5.0,4.0,1.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009,2008,31,North Carolina,ACC,2,1004,1,4,4,0,...,7.8,11.0,-2.0,-3.0,1.0,-0.0,1.0,1.0,-0.0,-3.0
2010,2008,37,Memphis,CUSA,13,1005,1,2,4,1,...,-5.5,-5.0,-2.0,-4.0,-5.0,-1.0,-1.0,1.0,0.0,0.0
2011,2008,13,UCLA,P10,23,1005,1,4,4,0,...,5.5,5.0,2.0,4.0,5.0,1.0,1.0,-1.0,-0.0,-0.0
2012,2008,43,Kansas,B12,7,1006,1,1,2,1,...,4.9,-2.6,8.0,11.0,7.0,6.0,3.0,2.0,2.0,10.0


## Model Testing

Choosing predictors and train test split

In [9]:
predictors = diff_team_stats.iloc[:,10:].columns
target = 'WIN'

#test on 2024 season
s24 = diff_team_stats.loc[diff_team_stats['YEAR']==2024,:]
x_s24 = s24[predictors]
y_s24 = s24[target]

#before 24 season
bf_24 = diff_team_stats.loc[diff_team_stats['YEAR']<2024]

#splitting on MATCHUP ID in order to ensure that the same matchups are split together
train_groups, test_groups = train_test_split(bf_24['MATCHUP ID'].unique(),test_size = .2, random_state=42)

x_train = bf_24[bf_24['MATCHUP ID'].isin(train_groups)][predictors]
y_train = bf_24[bf_24['MATCHUP ID'].isin(train_groups)][target]

x_test = bf_24[bf_24['MATCHUP ID'].isin(test_groups)][predictors]
y_test = bf_24[bf_24['MATCHUP ID'].isin(test_groups)][target]

Fitting on XGBoost Model as it usually tends to perform best on Tabular data but will try other models as well. Will also perform Cross Validation to find best hyper parameters.

In [120]:
parameter_grid = {
    'max_depth':np.arange(5, 11, 1),
    'min_child_weight':np.arange(0, 7, 1),
    'colsample_bytree':np.arange(0.5, 1.1, .1)
}

boost = XGBClassifier(n_estimators=1000,learning_rate=.0005,random_state=42)

grid_search = GridSearchCV(boost,param_grid=parameter_grid,cv=5,scoring='accuracy')

grid_search.fit(x_train,y_train)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
210 fits failed out of a total of 1470.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
210 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/core.py", line 726, in inner_f
    return func(**kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=0.000...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=1000,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             param_grid={'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1]),
                         'max_depth': array([ 5,  6,  7,  8,  9, 10]),
                         'min_child_weight': array([0, 1, 2, 3, 4, 5, 6])},
             scoring='accuracy')

In [10]:
best_xg = XGBClassifier(n_estimators=1000,learning_rate=.0005,colsample_bytree=0.6,min_child_weight=4,max_depth=6,random_state=42)
best_xg.fit(x_train,y_train)
pred = best_xg.predict(x_test)

In [26]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test.values,pred,labels=[1,0])

array([[142,  47],
       [ 46, 143]])

In [129]:
(142+143)/(142+47+46+143)

0.753968253968254

In [130]:
pred = best_xg.predict(x_s24)
confusion_matrix(y_s24.values,pred,labels=[1,0])

array([[44, 19],
       [19, 44]])

In [131]:
(44+44)/(44+44+19+19)

0.6984126984126984

In [37]:

def sim_tourney(df,model):
    rounds = [64,32,16,8,4,2]
    sim = {}
    
    features = df.iloc[:,10:].columns

    for rd in rounds:
        cur_rd = df.loc[df['CURRENT ROUND']==rd]
        
        x_cur_rd = cur_rd[features]
        
        cur_rd = cur_rd[['TEAM','CONF','CURRENT ROUND','WIN']]
        cur_rd['PRED WIN'] = model.predict(x_cur_rd)
        
        sim[f'{rd}'] = cur_rd
        
        next_rd = cur_rd.loc[cur_rd['PRED WIN']==1]
        next_rd = next_rd.assign(ROUND =rd / 2)
        
        df = df.drop(df.loc[df['ROUND'] == rd].index)  # Drop old round
        df = pd.concat([df, next_rd]) 
    return sim
        
simulated_24 = sim_tourney(s24,best_xg)


In [38]:
simulated_24['2']

,TEAM,CONF,CURRENT ROUND,WIN,PRED WIN
124,Connecticut,BE,2,1,1
125,Purdue,B10,2,0,0


Can see that the model predicted UConn to win!!

Trying a Random Grid Search to see if it has similar performance as previous search took 5 hrs to run.

In [24]:
parameter_grid = {
    'max_depth':np.arange(5, 11, 1),
    'min_child_weight':np.arange(0, 7, 1),
    'colsample_bytree':np.arange(0.5, 1.1, .1)
}

boost = XGBClassifier(n_estimators=200,learning_rate=.01,booster='gbtree',early_stopping_rounds=20,random_state=42)

ran_grid_search = RandomizedSearchCV(boost,param_distributions=parameter_grid,n_iter=50,cv=5,scoring='accuracy')

ran_rest = ran_grid_search.fit(x_train,y_train,eval_set=[(x_test, y_test)],verbose=False)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
50 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/core.py", line 726, in inner_f
    return func(**kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xg

In [28]:
print(ran_rest.best_params_)

best_ran_xg = XGBClassifier(n_estimators=200,learning_rate=.01,booster='gbtree',
                        colsample_bytree=0.6,min_child_weight=4,max_depth=5,random_state=42)
best_ran_xg.fit(x_train,y_train)
pred = best_xg.predict(x_test)

confusion_matrix(y_test.values,pred,labels=[1,0])

{'min_child_weight': 4, 'max_depth': 5, 'colsample_bytree': 0.6}


array([[142,  47],
       [ 46, 143]])

In [29]:
(142+143)/(142+143+47+46)

0.753968253968254

In [ ]:
sim = sim_tourney(s24,best_ran_xg)

In [69]:
sim['2']

,TEAM,CONF,CURRENT ROUND,WIN,PRED WIN
124,Connecticut,BE,2,1,1
125,Purdue,B10,2,0,0


## Optional: Trying adding supplemental features to model
Although from the results of the model it seems to do an incredible job at predicting the winner of each game let's see if we can slightly improve model performance by adding these supplemental data:
- Conference statistics
- Coach statistics 
- Upset data

Starting with Conference:

In [97]:
temp = comp_team_stats.copy()
temp = temp.drop_duplicates(subset=['YEAR', 'TEAM NO'],keep='first')
temp.loc[temp['YEAR']==2024,]


,YEAR,TEAM NO,TEAM,CONF,CONF ID,MATCHUP ID,SEED,ROUND,CURRENT ROUND,WIN,...,PAKE,PASE,R64,R32,S16,E8,F4,F2,CHAMP,TOP2
0,2024,1067,Connecticut,BE,8,0,1,1,64,1,...,10.8,13.3,10.0,6.0,5.0,5.0,5.0,4.0,4.0,2.0
1,2024,1026,Stetson,ASun,5,0,16,64,64,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024,1060,Florida Atlantic,Amer,4,1,8,64,64,0,...,2.8,2.7,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0
3,2024,1036,Northwestern,B10,6,1,9,32,64,1,...,0.9,0.8,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024,1029,San Diego St.,MWC,20,2,5,16,64,1,...,0.8,1.1,11.0,6.0,4.0,1.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2024,1079,Akron,MAC,17,29,14,64,64,0,...,-1.0,-1.3,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
60,2024,1023,Texas,B12,7,30,7,32,64,1,...,-5.6,-4.5,13.0,7.0,2.0,2.0,0.0,0.0,0.0,2.0
61,2024,1068,Colorado St.,MWC,20,30,10,64,64,0,...,-0.9,-2.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
62,2024,1024,Tennessee,SEC,28,31,2,8,64,1,...,-2.0,-0.3,11.0,8.0,6.0,2.0,0.0,0.0,0.0,3.0


In [108]:
temp = comp_team_stats.copy()
temp = temp.drop_duplicates(subset=['YEAR', 'TEAM NO'],keep='first')
temp = temp.drop(['TEAM NO','TEAM','CONF','MATCHUP ID','CURRENT ROUND'],axis=1)
grouped = temp.groupby(['YEAR','CONF ID'])
grouped = grouped.mean()

In [114]:
conf_perf[['CONF','CONF ID']]

,CONF,CONF ID
0,A10,1
1,ACC,2
2,AE,3
3,Amer,4
4,ASun,5
5,B10,6
6,B12,7
7,BE,8
8,BSky,9
9,BSth,10


In [111]:
grouped.loc[2024]

,SEED,ROUND,WIN,K OFF,K DEF,AVG HGT,EFF HGT,BADJ O_A,BADJ D_A,WIN%_A,...,PAKE,PASE,R64,R32,S16,E8,F4,F2,CHAMP,TOP2
CONF ID,,,,,,,,,,,,,,,,,,,,,
1,9.000000,32.000000,1.000000,109.958500,101.014250,76.719000,80.412000,112.400000,98.800000,50.000000,...,1.750000,0.800000,3.500000,2.500000,0.500,0.500000,0.000000,0.000000,0.000000,0.000000
2,5.500000,9.000000,1.000000,113.767750,101.663925,77.693250,81.358500,115.900000,96.975000,57.500000,...,3.950000,2.950000,10.750000,7.750000,6.250,4.000000,2.250000,1.500000,1.000000,5.000000
3,13.000000,64.000000,0.000000,107.533000,97.234900,76.920000,79.286000,105.500000,101.600000,71.428571,...,-1.800000,-1.100000,7.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000
4,10.000000,64.000000,0.000000,114.225500,106.512500,76.815000,80.289500,114.350000,108.700000,54.166667,...,1.550000,1.250000,2.500000,1.000000,0.500,0.500000,0.500000,0.000000,0.000000,0.000000
5,16.000000,64.000000,0.000000,111.410000,112.757000,77.673000,81.379000,106.000000,115.400000,37.500000,...,0.000000,0.000000,1.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000
6,5.833333,33.666667,0.666667,114.017167,102.320267,77.930167,81.307333,117.083333,99.350000,40.530303,...,0.716667,1.016667,9.166667,7.000000,4.000,1.666667,1.166667,0.500000,0.000000,1.666667
7,4.750000,40.000000,0.625000,112.370125,98.259512,77.535250,80.744000,116.887500,98.912500,44.873737,...,-1.362500,-1.462500,9.500000,6.625000,3.625,2.000000,0.875000,0.625000,0.375000,2.500000
8,2.000000,11.000000,1.000000,117.172333,99.621300,78.318667,81.933000,120.633333,97.366667,64.957265,...,3.266667,3.966667,10.000000,6.666667,4.000,2.333333,1.666667,1.333333,1.333333,1.333333
10,16.000000,64.000000,0.000000,107.318000,101.377000,77.071000,79.691000,103.900000,106.500000,37.500000,...,-0.100000,-0.200000,2.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000


In [44]:
conf_perf = conf_perf.drop(['GAMES','W','L'],axis=1)
conf_perf.head()

,CONF ID,CONF,PAKE,PAKE RANK,PASE,PASE RANK,WIN%,R64,R32,S16,E8,F4,F2,CHAMP,TOP2,CHAMP%
0,1,A10,-1.5,24,-5.8,29,0.400,48,23,7,2,0,0,0,0,9.00%
1,2,ACC,17.5,1,12.4,1,0.636,95,65,43,25,12,7,5,27,95.30%
2,3,AE,-1.8,25,-0.9,24,0.059,16,1,0,0,0,0,0,0,0.00%
3,4,Amer,-5.3,30,-0.6,21,0.540,24,14,6,3,2,1,1,3,51.20%
4,5,ASun,0.2,13,1.2,11,0.200,16,3,1,0,0,0,0,0,0.50%


Next what came out to be much more labor intensive due to the nature of the data, Coaching:

Starting with cleaning up the provided coach stats dataset, either due to typos or column values not being integers

In [46]:
#converting F4% and CHAMP% as a int
coach_perf["F4%"] = coach_perf["F4%"].str.rstrip("%").astype(float) / 100
coach_perf["CHAMP%"] = coach_perf["CHAMP%"].str.rstrip("%").astype(float) / 100


steve_prohm = {
    "COACH ID": min(coach_perf.iloc[72,0],coach_perf.iloc[187,0],coach_perf.iloc[291,0]),  # Keep the lower ID
    "COACH": "Steve Prohm",  # Corrected name
    "PAKE": coach_perf.loc[[72, 187, 291], "PAKE"].mean(),
    "PAKE RANK": coach_perf.loc[[72, 187, 291], "PAKE RANK"].mean(),
    "PASE": coach_perf.loc[[72, 187, 291], "PASE"].mean(),
    "PASE RANK": coach_perf.loc[[72, 187, 291], "PASE RANK"].mean(),
    "GAMES": coach_perf.loc[[72, 187, 291], "GAMES"].sum(),
    "W": coach_perf.loc[[72, 187, 291], "W"].sum(),
    "L": coach_perf.loc[[72, 187, 291], "L"].sum(),
    "WIN%": coach_perf.loc[[72, 187, 291], "WIN%"].mean(),
    "R64": coach_perf.loc[[72, 187, 291], "R64"].sum(),
    "R32": coach_perf.loc[[72, 187, 291], "R32"].sum(),
    "S16": coach_perf.loc[[72, 187, 291], "S16"].sum(),
    "E8": coach_perf.loc[[72, 187, 291], "E8"].sum(),
    "F4": coach_perf.loc[[72, 187, 291], "F4"].sum(),
    "F2": coach_perf.loc[[72, 187, 291], "F2"].sum(),
    "CHAMP": coach_perf.loc[[72, 187, 291], "CHAMP"].sum(),
    "TOP2": coach_perf.loc[[72, 187, 291], "TOP2"].sum(),
    "F4%": coach_perf.loc[[72, 187, 291], "F4%"].mean(),
    "CHAMP%": coach_perf.loc[[72, 187, 291], "CHAMP%"].mean(),
}

bob_marlin = {
    "COACH ID": min(coach_perf.iloc[164,0],coach_perf.iloc[215,0]),  # Keep the lower ID
    "COACH": "Bob Marlin",  # Corrected name
    "PAKE": coach_perf.loc[[164, 215], "PAKE"].mean(),
    "PAKE RANK": coach_perf.loc[[164, 215], "PAKE RANK"].mean(),
    "PASE": coach_perf.loc[[164, 215], "PASE"].mean(),
    "PASE RANK": coach_perf.loc[[164, 215], "PASE RANK"].mean(),
    "GAMES": coach_perf.loc[[164, 215], "GAMES"].sum(),
    "W": coach_perf.loc[[164, 215], "W"].sum(),
    "L": coach_perf.loc[[164, 215], "L"].sum(),
    "WIN%": coach_perf.loc[[164, 215], "WIN%"].mean(),
    "R64": coach_perf.loc[[164, 215], "R64"].sum(),
    "R32": coach_perf.loc[[164, 215], "R32"].sum(),
    "S16": coach_perf.loc[[164, 215], "S16"].sum(),
    "E8": coach_perf.loc[[164, 215], "E8"].sum(),
    "F4": coach_perf.loc[[164, 215], "F4"].sum(),
    "F2": coach_perf.loc[[164, 215], "F2"].sum(),
    "CHAMP": coach_perf.loc[[164, 215], "CHAMP"].sum(),
    "TOP2": coach_perf.loc[[164, 215], "TOP2"].sum(),
    "F4%": coach_perf.loc[[164, 215], "F4%"].mean(),
    "CHAMP%": coach_perf.loc[[164, 215], "CHAMP%"].mean(),
}

kermit_davis = {
    "COACH ID": min(coach_perf.iloc[54,0],coach_perf.iloc[193,0]),  # Keep the lower ID
    "COACH": "Kermit Davis",  # Corrected name
    "PAKE": coach_perf.loc[[54, 193], "PAKE"].mean(),
    "PAKE RANK": coach_perf.loc[[54, 193], "PAKE RANK"].mean(),
    "PASE": coach_perf.loc[[54, 193], "PASE"].mean(),
    "PASE RANK": coach_perf.loc[[54, 193], "PASE RANK"].mean(),
    "GAMES": coach_perf.loc[[54, 193], "GAMES"].sum(),
    "W": coach_perf.loc[[54, 193], "W"].sum(),
    "L": coach_perf.loc[[54, 193], "L"].sum(),
    "WIN%": coach_perf.loc[[54, 193], "WIN%"].mean(),
    "R64": coach_perf.loc[[54, 193], "R64"].sum(),
    "R32": coach_perf.loc[[54, 193], "R32"].sum(),
    "S16": coach_perf.loc[[54, 193], "S16"].sum(),
    "E8": coach_perf.loc[[54, 193], "E8"].sum(),
    "F4": coach_perf.loc[[54, 193], "F4"].sum(),
    "F2": coach_perf.loc[[54, 193], "F2"].sum(),
    "CHAMP": coach_perf.loc[[54, 193], "CHAMP"].sum(),
    "TOP2": coach_perf.loc[[54, 193], "TOP2"].sum(),
    "F4%": coach_perf.loc[[54, 193], "F4%"].mean(),
    "CHAMP%": coach_perf.loc[[54, 193], "CHAMP%"].mean(),
}

coach_perf = coach_perf.drop([54, 72, 164, 187, 193, 215, 291])

# Reset index
coach_perf = coach_perf.reset_index(drop=True)

# Add the new combined row
coach_perf = pd.concat([coach_perf, pd.DataFrame([steve_prohm,kermit_davis,bob_marlin])], ignore_index=True)

coach_perf

,COACH ID,COACH,PAKE,PAKE RANK,PASE,PASE RANK,GAMES,W,L,WIN%,R64,R32,S16,E8,F4,F2,CHAMP,TOP2,F4%,CHAMP%
0,1,Roy Williams,8.200000,1.000000,5.50,6.000000,44,34,10,0.7730,12,11,8,6,4,3,2,8,0.955000,0.5380
1,2,Tom Izzo,8.000000,2.000000,9.60,1.000000,47,32,15,0.6810,15,13,9,5,4,1,0,4,0.867000,0.3420
2,3,Brad Stevens,6.500000,3.000000,7.60,3.000000,17,12,5,0.7060,5,4,2,2,2,2,0,0,0.255000,0.0370
3,4,John Calipari,6.200000,4.000000,8.60,2.000000,52,39,13,0.7500,14,12,10,8,5,3,1,8,0.990000,0.7490
4,5,Dana Altman,6.100000,5.000000,6.70,4.000000,24,16,8,0.6670,8,8,5,2,1,0,0,1,0.431000,0.0820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,318,Rick Barnes,-6.400000,318.000000,-3.80,313.000000,28,15,13,0.5360,13,9,4,2,0,0,0,3,0.809000,0.2810
311,319,Tony Bennett,-6.500000,319.000000,-7.00,318.000000,24,15,9,0.6250,10,6,4,2,1,1,1,5,0.936000,0.5260
312,73,Steve Prohm,-0.466667,184.333333,-0.30,169.666667,8,4,4,0.5000,4,3,1,0,0,0,0,0,0.083333,0.0120
313,55,Kermit Davis,0.250000,124.500000,0.35,118.500000,5,2,3,0.4165,3,2,0,0,0,0,0,0,0.011000,0.0005


This data is missing what years and teams a coach was coaching in order to merge onto the team statisitics dataset. Below is the painstaking process of manually entering the correct information for this dataset:

In [47]:
coach_perf['TEAMS'] = [['North Carolina'],['Michigan St.'],['Butler'],['Memphis','Kentucky','Arkansas'],['Creighton','Oregon'],['Syracuse'],['Michigan'],
                      ['Villanova'],['Nevada','Arkansas','USC'],['Connecticut'],['Loyola Chicago','Oklahoma'],['Kansas St.','South Carolina','Massachusetts'],
                      ['Wagner','Rhode Island','Connecticut'],['Kansas'],['George Mason','Miami FL'],['North Carolina'],['Gonzaga'],['Little Rock','Texas Tech','Texas','Missisippi'],
                      ['Florida'],['Connecticut',"Saint Joseph's"],["Saint Peter's",'Seton Hall'],['Florida Atlantic','Michigan'],['Louisville','Iona',"Saint John's"],['Duke'],
                      ['Xavier','Arizona','Xavier'],['Wisconsin'],['Ohio','Illinois','Akron'],['Dayton','Indiana','Rhode Island'],['San Diego St.'],
                      ['Oklahoma','Pittsburgh'],['Florida Gulf Coast','USC','SMU'],['Oral Robert','Wichita St.'],['Michigan'],['Stanford','UCF'],['Murray St.','Texas A&M'],['La Salle'],
                      ['Kansas St.'],['UNC Wilmington','North Carolina St.'],['Western Kentucky','South Carolina','Northern Kentucky'],['Princeton'],['Florida St.'],['Grand Canyon'],['Richmond'],
                      ['Montana','Oregon St.'],['Florida','Georgia'],['Buffalo','Alabama'],['Xavier','Louisville','College of Charleston'],['Alabama','North Carolina St.','Cal St. Northridge'],
                      ['Cornell','Boston College','Penn'],['Fairleigh Dickinson','Iona'],['Wright St.','Clemson'],['Northwestern'],['Norfolk St.'],['Mercer'],['UAB','Stanford'],
                      ['Yale'],['Marshall'],['UC Irvine'],['Morehead St.','Southern Miss','Tennessee'],['Western Kentucky'],['Abilene Christian','UTEP'],['Dayton','Georgia Tech','South Florida'],
                      ['San Diego'],['Furman'],['Duke'],['James Madison','Vanderbilt'],['Iowa St.','Creighton'],['Murray St.','LSU'],['Stony Brook','Ohio'],['Stanford','LSU','TCU'],
                      ['Northern Iowa'],['Lehigh'],['Alabama'],['Cleveland St.','Missouri'],['Washington'],['Arkansas'],
                      ['Hawaii'],['Cleveland St.'],['Georgia St.'],['Liberty','Liberty'],['Washington'],['Penn St.','Notre Dame'],['Oakland'],['Marquette','Virginia Tech','Texas A&M'],
                      ['Frenso St.','UTEP','Texas'],['Washington St.','Stanford'],['Saint Louis'],['North Dakota St.','Ohio'],['Georgia Tech','George Mason'],['Harvard'],['Baylor'],
                      ['Missouri','Arkansas',"Saint John's"],['Stony Brook','Rutgers'],['Montana St.','Utah St.','Washington'],['South Florida','Eastern Michigan'],['Utah'],
                      ['Chattanooga','Central Arkansas'],['UTSA'],['UT Arlington','Troy'],['Hampton'],['Mississippi St.','Western Kentucky'],['Arkansas Pine Bluff','Mississippi Valley St.'],
                      ['Lafayette'],['Southern Miss','Colorado St.'],['Mississippi Valley St.'],['Northwestern','Holy Cross'],['North Carolina A&T'],['Robert Morris'],['Radford'],
                      ['Grambling St.'],['Wagner'],['Marshall','UCF','Stetson'],['Fairleigh Dickinson'],['Bradley','UC Davis'],['Gardner Webb'],['Hartford','Manhattan'],
                      ['Radford','UNC Greensboro'],['North Dakota St.'],["Mount St. Mary's",'Siena','George Washington'],['Wagner',"Saint Peter's"],['UNC Asheville'],['Bradley'],
                      ['Norfolk St.'],['Georgia St.','SMU','Rice'],['Howard'],['Texas Southern'],['Army','Drexel'],['Stetson','Lipscomb','Belmont'],['Milwaukee','Butler'],
                      ['UMBC','Utah St.','VCU'],['North Dakota'],['Texas A&M','Maryland'],['Southern'],['Troy'],['Cal Poly'],['UNC Asheville','Middle Tennessee'],['Detroit'],
                      ['Cal St. Northridge'],['Binghamton','Morgan St.'],['James Madison'],['North Dakota St.'],['UMBC'],['Penn St.','Florida Gulf Coast'],['Northwestern St.'],
                      ['Boise St.'],['Austin Peay'],['Hampton'],['Loyola MD'],['Coastal Carolina'],["Mount St. Mary's",'Holy Cross'],['Northern Colorado'],['Milwaukee'],['Longwood'],
                      ['Delaware'],['Northern Kentucky','Cincinnati'],['Cal St. Fullerton'],['Stephen F. Austin'],['Notre Dame'],['Akron','Duquesne'],['Texas A&M Corpus Chris','Western Kentucky','Oklahoma St.'],
                      ['East Tennessee St.','Wake Forest'],['Maryland'],['Eastern Kentucky'],['East Tennesse St.'],['Cal St. Fullerton'],['Pacific'],['Princeton','Fairfield'],['Manhattan'],
                      ['Winthrop'],['Florida Southern','Green Bay'],['Eastern Washington'],['Georgia St.','Cal St. Bakersfield'],['Chattanooga','Massachusetts'],['UNC Asheville'],['Western Michigan'],
                      ['Indiana St.'],['Delaware','North Carolina A&T'],['Wyoming'],['UC Irvine'],['Robert Morris','Rutgers'],['Stephen F. Austin'],['Morgan St.'],["Saint Peter's"],['American'],
                      ['New Mexico St.','New Mexico'],['Charleston','Boston College'],['Kennesaw St.','South Florida'],['Oklahoma St.'],['Montana'],['Morehead St.'],['Tennessee','Auburn'],['Northeastern'],
                      ['Georgia','Cleveland St.'],['North Carolina Central'],['Army','Saint Louis'],['Houston'],['Tulsa','Wake Forest'],['Kentucky','Texas Tech','Ranger College','Tarleton St.'],['Portland St.','Washington St.'],
                      ['Weber St.'],['UC Santa Barbara'],['California'],['Long Island','Dusquesne','UMBC'],['Samford'],['Western Kentucky','Jacksonville St.'],['Eastern Washington','Portland'],['Bucknell','New Hampshire'],
                      ['Florida Gulf Coast','East Carolina'],['Midland College','Midwestern St.','Arkansas St.','North Texas','Texas Tech'],['UNC Greensboro','Cincinnati'],['American','Old Dominion'],
                      ['Boston College','Kennesaw St.'],['Georgetown'],['Oral Roberts'],['South Alabama'],['Massachusetts','LIU Brooklyn'],['Albany'],['UNLV','Oklahoma'],['Old Dominion'],['South Dakota St.'],['New Mexico','UCLA','Nevada'],
                      ['Mississippi','UAB'],['Indiana'],['Texas Tech'],['UC Santa Barbara'],['Kent St.'],['Kent St.','TCU','Ohio','Boston College'],['UTEP','Auburn','Central Michigan'],['Oregon','Washington St.'],
                      ['Wake Forest'],['Wisconsin'],['Minnesota','Long Beach St.','Eastern Washington'],['Loyola Chicago'],['Vermont','George Washington'],['USC'],['Penn St.','Navy'],['Marquette','Indiana','Georgia'],
                      ['Iona'],['South Dakota St.','UNLV','Iowa St.'],['North Texas','LSU','Texas Southern'],['Wichita St.'],['Bowling Green','New Mexico St.','Mississippi St.'],['Indiana'],['San Diego St.'],['UCLA','Mississippi St.'],
                      ['Furman','Drake','Colorado St.'],['Davidson'],['USC','UTEP'],["Saint Joseph's"],['Bucknell','George Mason','Holy Cross'],['Nevada','Georgia','California'],['Utah'],['Memphis'],['Rice','VCU','Penn St.'],
                      ['Colgate'],['SMU'],['Colorado St.','Nebraska','San Jose St.'],['New Mexico'],['Northen Colorado','Colorado'],['VCU','Alabama','Dayton'],['Chattanooga','South Carolina'],['South Dakota St.','Wright St.'],
                      ['Winthrop','Charleston','Louisville'],['FIU','Minnesota','New Mexico'],['Boise St.'],['Arizona St.','Santa Clara'],['West Virginia'],['Memphis','Georgia Tech'],['Fairfield','Providence','Georgetown'],['Valparaiso','Vanderbilt','Grand Canyon'],
                      ['South Dakota','Utah St.','Utah'],['Indiana','Houston'],["Saint John's",'San Diego'],['Drake','Providence','Central Michigan'],['Butler','Ohio St.','Depaul'],['Chattanooga','VCU','LSU','McNeese St.'],
                      ['Cincinnati','UCLA'],['Drake','West Virginia'],['Stephen F. Austin','Oklahoma St.','Illinois'],['New Mexico St.','UNLV'],['Marquette'],['Buffalo','Arizona St.'],['SMU'],['Iowa St.','Nebraska'],
                      ['St. Bonaventure'],['Vermont'],['Minnesota','Texas Tech','Memphis'],['UNLV'],['San Francisco','Florida'],['Tennessee','California','Missouri'],['Iona','Seton Hall','Maryland'],
                      ['Utah St.'],['Ohio St.','Butler'],['BYU','Kentucky'],['Illinois','Kansas St.'],['Belmont'],['Miami FL','Missouri','Tulsa'],['BYU'],['Purdue'],['Temple','La Salle'],
                      ['Siena','Iowa'],['Vanderbilt','Pittsburgh'],['Wofford','Virginia Tech'],['VCU','Texas','Marquette'],['Arizona'],['Oklahoma St.','Saint Louis'],['Clemson'],["Saint Mary's"],['Pittsburgh','TCU'],['Georgetown'],['Texas','Tennessee'],['Washington St.','Virginia']
                      ,['Murrary St.','Iowaw St.','Murrary St.'],['Middle Tennessee','Mississippi'],['Sam Houston St.','Louisiana Lafayette'],]
coach_perf['YRS'] = [['2003-2021'],['1995-2025'],['2007-2013'],['2000-2009','2009-2024','2024-2025'],['1994-2010','2010-2025'],['1976-2023'],['2007-2019'],
                     ['2001-2022'],['2015-2019','2019-2024','2024-2025'],['2012-2018'],['2011-2021','2021-2025'],['2007-2012','2012-2022','2022-2025'],
                     ['2010-2012','2012-2018','2018-2025'],['2003-2025'],['1997-2011','2011-2024'],['2021-2025'],['1999-2025'],['2015-2016','2016-2021','2021-2022','2023-2025'],
                     ['1996-2015'],['1986-2012','2018-2021'],['2018-2022','2022-2025'],['2018-2024','2024-2025'],['2001-2017','2020-2023','2024-2025'],['1980-2022'],
                     ['2004-2009','2009-2021','2022-2025'],['2001-2015'],['2008-2012','2012-2017','2017-2025'],['2011-2017','2017-2021','2022-2025'],['2017-2025'],
                     ['2006-2011','2018-2025'],['2011-2013','2013-2024','2024-2025'],['2017-2023','2023-2025'],['2019-2024'],['2008-2016','2016-2025'],['2006-2011','2011-2019'],['2004-2018'],
                     ['2022-2025'],['2014-2017','2017-2025'],['2003-2008','2008-2012','2019-2025'],['2011-2025'],['2002-2025'],['2009-2013'],['2005-2025'],
                     ['2006-2014','2014-2025'],['2015-2022','2022-2025'],['2015-2019','2019-2025'],['2009-2018','2018-2022','2024-2025'],['1998-2009','2011-2017','2018-2021'],
                     ['2000-2010','2010-2014','2015-2025'],['2022-2023','2023-2025'],['2006-2010','2010-2025'],['2013-2025'],['2007-2013'],['2008-2019'],['2012-2016','2016-2024'],
                     ['1999-2025'],['2014-2024'],['2010-2025'],['2006-2012','2012-2014','2014-2015'],['2008-2012'],['2011-2021','2021-2025'],['2003-2011','2011-2016','2017-2023'],
                     ['2007-2015'],['2017-2025'],['2022-2025'],['2020-2024','2024-2025'],['2006-2010','2010-2025'],['2015-2022','2022-2025'],['2016-2019','2019-2025'],['2004-2008','2008-2012','2012-2016'],
                     ['2006-2025'],['2007-2025'],['2015-2019'],['2019-2022','2022-2025'],['2017-2024'],['2007-2011'],
                     ['2015-2025'],['2006-2017'],['2011-2019'],['2007-2009','2015-2025'],['2002-2017'],['2021-2023','2023-2025'],['1984-2025'],['2008-2014','2014-2019','2019-2025'],
                     ['2011-2018','2018-2021','2022-2025'],['2019-2024','2024-2025'],['2007-2012'],['2007-2014','2014-2019'],['2000-2011','2011-2015'],['2007-2025'],['2003-2025'],
                     ['2006-2011','2011-2019','2019-2023'],['2005-2016','2016-2025'],['2019-2020','2023-2024','2024-2025'],['2007-2014','2021-2025'],['2011-2021'],
                     ['2004-2013','2024-2025'],['2006-2016'],['2006-2018','2019-2025'],['2009-2024'],['1998-2012','2016-2023'],['2008-2021','2022-2025'],
                     ['1995-2022'],['2004-2012','2012-2018'],['2005-2008'],['2000-2013','2015-2019'],['2012-2016'],['2010-2025'],['2007-2011'],
                     ['2017-2025'],['2022-2025'],['2007-2010','2010-2016','2019-2025'],['2013-2022'],['2002-2011','2011-2025'],['2013-2024'],['2010-2022','2023-2025'],
                     ['2011-2021','2021-2025'],['2014-2025'],['2012-2018','2018-2019','2019-2022'],['2012-2022','2022-2025'],['2018-2025'],['2015-2025'],
                     ['2014-2025'],['2019-2022','2022-2024','2024-2025'],['2019-2025'],['2012-2018'],['2009-2016','2016-2025'],['2011-2013','2013-2019','2019-2025'],['2016-2017','2017-2022'],
                     ['2016-2021','2021-2023','2023-2025'],['2006-2019'],['2007-2011','2011-2021'],['2011-2017'],['2013-2019'],['2009-2019'],['2013-2018','2018-2025'],['2008-2016'],
                     ['1996-2013'],['2007-2009','2019-2025'],['2008-2016'],['2014-2025'],['2004-2012'],['2011-2020','2022-2025'],['1999-2022'],
                     ['2002-2010'],['1990-2017'],['2009-2024'],['2004-2013'],['2007-2023'],['2003-2010','2010-2015'],['2010-2016'],['2005-2016'],['2018-2025'],
                     ['2016-2025'],['2015-2019','2019-2021'],['2013-2025'],['2016-2025'],['2000-2023'],['2004-2017','2017-2024'],['2021-2023','2023-2024','2024-2025'],
                     ['2015-2020','2020-2025'],['1989-2011'],['2005-2015'],['2003-2015'],['2003-2012'],['1988-2013'],['2007-2011','2011-2019'],['2011-2022'],
                     ['2007-2012'],['2006-2015','2015-2020'],['2011-2017'],['2007-2011','2011-2025'],['2015-2017','2017-2022'],['1996-2013'],['2003-2020'],
                     ['2010-2021'],['2006-2016','2023-2025'],['2011-2016'],['2010-2025'],['2007-2010','2010-2013'],['2000-2013'],['2006-2019'],['2006-2018'],['2013-2023'],
                     ['2016-2017','2017-2021'],['2014-2021','2021-2025'],['2019-2023','2023-2024'],['2017-2024'],['2014-2025'],['2016-2024'],['2005-2011','2014-2025'],['2006-2025'],
                     ['2003-2009','2017-2019'],['2009-2025'],['2002-2009','2012-2016'],['2004-2010'],['2012-2014','2014-2020'],['2007-2009','2011-2012','2015-2020','2020-2025'],['2005-2009','2009-2014'],
                     ['2006-2022'],['1998-2017'],['2008-2014'],['2002-2012','2012-2017','2021-2025'],['2020-2025'],['2012-2016','2016-2025'],['2017-2021','2021-2025'],['2015-2023','2023-2025'],
                     ['2013-2018','2018-2022'],['2004-2009','2016-2017','2017-2023','2023-2025'],['2011-2021','2021-2025'],['2000-2013','2013-2024'],
                     ['1997-2010','2015-2019'],['2017-2023'],['1999-2017'],['2007-2012'],['2008-2017','2017-2019'],['2002-2021'],['2004-2011','2011-2021'],['2001-2013'],['2019-2025'],['2007-2013','2013-2018','2019-2025'],
                     ['2006-2018','2020-2025'],['2021-2025'],['2021-2023'],['2017-2025'],['2011-2025'],['2002-2008','2008-2012','2012-2014','2014-2021'],['2006-2010','2010-2014','2021-2025'],['1997-2010','2014-2019'],
                     ['2007-2010'],['2015-2025'],['1999-2006','2007-2024','2024-2025'],['2021-2025'],['2005-2011','2011-2016'],['2009-2013'],['2003-2011','2011-2025'],['1999-2008','2008-2017','2018-2022'],
                     ['2010-2019'],['2016-2019','2019-2021','2021-2025'],['2001-2012','2012-2017','2018-2025'],['2007-2020'],['2014-2015','2017-2022','2022-2025'],['2007-2008'],['1999-2017'],['2003-2013','2015-2022'],
                     ['2013-2017','2017-2018','2018-2025'],['1989-2022'],['2005-2009','2010-2017'],['1995-2019'],['2008-2015','2015-2021','2023-2025'],['2004-2009','2009-2018','2019-2023'],['2007-2011'],['2018-2025'],['2014-2017','2017-2023','2023-2025'],
                     ['2011-2025'],['2012-2016'],['2007-2012','2012-2019','2021-2025'],['2013-2017'],['2006-2010','2010-2025'],['2006-2009','2009-2015','2017-2025'],['2017-2022','2022-2025'],['1995-2016','2016-2024'],
                     ['2012-2021','2021-2024','2024-2025'],['2012-2013','2013-2021','2021-2025'],['2010-2025'],['2006-2015','2016-2025'],['2007-2023'],['2009-2016','2016-2023'],['2006-2011','2011-2023','2023-2025'],['2011-2016','2016-2019','2020-2025'],
                     ['2014-2018','2018-2021','2021-2025'],['2006-2008','2014-2025'],['2010-2015','2022-2025'],['2007-2008','2008-2011','2012-2021'],['2014-2017','2017-2024','2024-2025'],['2013-2015','2015-2017','2017-2022','2023-2025'],
                     ['2006-2019','2019-2025'],['2018-2024','2024-2025'],['2013-2016','2016-2017','2017-2025'],['2007-2016','2016-2019'],['2014-2021'],['2013-2015','2015-2025'],['2016-2022'],['2010-2015','2019-2025'],
                     ['2007-2025'],['2011-2025'],['2007-2013','2013-2016','2016-2018'],['2011-2016'],['2019-2022','2022-2025'],['2011-2014','2014-2017','2017-2022'],['2007-2010','2010-2022','2022-2025'],
                     ['1998-2015'],['2004-2017','2022-2025'],['2019-2024','2024-2025'],['2003-2012','2012-2022'],['1986-2019'],['2004-2011','2011-2014','2014-2022'],['2005-2019'],['2005-2025'],['2006-2019','2022-2025'],
                     ['2005-2010','2010-2025'],['1999-2016','2016-2018'],['2002-2019','2019-2025'],['2009-2015','2015-2021','2021-2025'],['2021-2025'],['2008-2016','2016-2024'],['2003-2010'],['2001-2025'],['2003-2016','2016-2025'],['2004-2017'],['1998-2015','2015-2025'],['2006-2009','2009-2024']
                     ,['2011-2015','2015-2021','2022-2025'],['2002-2018','2018-2023'],['1998-2010','2010-2024']]


In [48]:
coach_perf.head()

,COACH ID,COACH,PAKE,PAKE RANK,PASE,PASE RANK,GAMES,W,L,WIN%,...,S16,E8,F4,F2,CHAMP,TOP2,F4%,CHAMP%,TEAMS,YRS
0,1,Roy Williams,8.2,1.0,5.5,6.0,44,34,10,0.773,...,8,6,4,3,2,8,0.955,0.538,[North Carolina],[2003-2021]
1,2,Tom Izzo,8.0,2.0,9.6,1.0,47,32,15,0.681,...,9,5,4,1,0,4,0.867,0.342,[Michigan St.],[1995-2025]
2,3,Brad Stevens,6.5,3.0,7.6,3.0,17,12,5,0.706,...,2,2,2,2,0,0,0.255,0.037,[Butler],[2007-2013]
3,4,John Calipari,6.2,4.0,8.6,2.0,52,39,13,0.750,...,10,8,5,3,1,8,0.990,0.749,"[Memphis, Kentucky, Arkansas]","[2000-2009, 2009-2024, 2024-2025]"
4,5,Dana Altman,6.1,5.0,6.7,4.0,24,16,8,0.667,...,5,2,1,0,0,1,0.431,0.082,"[Creighton, Oregon]","[1994-2010, 2010-2025]"


Need to create a function that'll pull the correct TEAM NO depending on the newly provided columns

In [ ]:
def 